In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests5

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


# a  = .25, k= 4, something not right?

In [3]:
a = 0.25
num_params=4
num_sims = 100

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.96 & 0.94 & 0.97 & 0.99   \\
Model 1 & 0.02 & 0.02 & 0.03 & 0.05 & 0.03 & 0.01   \\
Model 2 & 0.03 & 0.03 & 0.01 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.95, 0.02, 0.03]), array([0.95, 0.02, 0.03]), array([0.96, 0.03, 0.01]), array([0.94, 0.05, 0.01]), array([0.97, 0.03, 0.  ]), array([0.99, 0.01, 0.  ]), 0.8488602649149568, 5.742349796505308, 5.2099405125670355)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.96 & 0.97 & 0.94 & 0.99   \\
Model 1 & 0.03 & 0.03 & 0.04 & 0.03 & 0.05 & 0.01   \\
Model 2 & 0.01 & 0.01 & 0.00 & 0.00 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.96, 0.03, 0.01]), array([0.96, 0.03, 0.01]), array([0.96, 0.04, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.94, 0.05, 0.01]), array([0.99, 0.01, 0.  ]), 3.1879670510537563, 6.413646098274674, 6.996725060424468)


# a  = .25, k= 9

In [6]:
a = 0.25
num_params=9
num_sims = 100

In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.97 & 0.97 & 0.97 & 0.98   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.02 & 0.02 & 0.02   \\
Model 2 & 0.06 & 0.06 & 0.01 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.93, 0.01, 0.06]), array([0.93, 0.01, 0.06]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.98, 0.02, 0.  ]), -2.684095638776855, 4.961967636959473, 5.645374648477944)


In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.94 & 0.94 & 0.94 & 0.98   \\
Model 1 & 0.01 & 0.01 & 0.06 & 0.05 & 0.05 & 0.02   \\
Model 2 & 0.02 & 0.02 & 0.00 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), array([0.94, 0.06, 0.  ]), array([0.94, 0.05, 0.01]), array([0.94, 0.05, 0.01]), array([0.98, 0.02, 0.  ]), -1.0985999115826912, 7.8951865432907855, 7.873216133246676)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.98 & 0.98 & 0.98 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.08 & 0.08 & 0.02 & 0.01 & 0.02 & 0.01   \\
\hline
\end{tabular}
(array([0.92, 0.  , 0.08]), array([0.92, 0.  , 0.08]), array([0.98, 0.  , 0.02]), array([0.98, 0.01, 0.01]), array([0.98, 0.  , 0.02]), array([0.99, 0.  , 0.01]), -3.5106779377062862, 4.071044809147577, 4.282369115210032)


# a  = 0, k= 9

In [10]:
a = 0.00 
num_params=9
num_sims = 100

In [11]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.97 & 0.99 & 0.97 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.03 & 0.01 & 0.03 & 0.01   \\
Model 2 & 0.02 & 0.02 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.98, 0.  , 0.02]), array([0.98, 0.  , 0.02]), array([0.97, 0.03, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.99, 0.01, 0.  ]), -3.4107995423513904, 1.9979417733559255, 2.7950479141178617)


In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.97 & 0.99 & 0.99 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.03 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.02 & 0.02 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.98, 0.  , 0.02]), array([0.98, 0.  , 0.02]), array([0.97, 0.03, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), -3.3257068826768874, 2.213662414587975, 2.8201487922253774)


# a  = .25, K = 19

In [13]:
a = 0.25
num_params=19
num_sims = 100

In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.73 & 0.73 & 0.99 & 0.98 & 0.99 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.27 & 0.27 & 0.01 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.73, 0.  , 0.27]), array([0.73, 0.  , 0.27]), array([0.99, 0.  , 0.01]), array([0.98, 0.01, 0.01]), array([0.99, 0.  , 0.01]), array([1., 0., 0.]), -9.549747640604519, 5.683417753641636, 6.759046723459051)


In [15]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.85 & 0.85 & 0.97 & 0.95 & 0.97 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.03 & 0.01 & 0.02   \\
Model 2 & 0.15 & 0.15 & 0.01 & 0.02 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.85, 0.  , 0.15]), array([0.85, 0.  , 0.15]), array([0.97, 0.02, 0.01]), array([0.95, 0.03, 0.02]), array([0.97, 0.01, 0.02]), array([0.98, 0.02, 0.  ]), -7.9721585972270805, 7.878111872166247, 8.515159897779391)


In [16]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.94 & 0.93 & 0.94 & 0.94   \\
Model 1 & 0.02 & 0.02 & 0.06 & 0.07 & 0.05 & 0.06   \\
Model 2 & 0.05 & 0.05 & 0.00 & 0.00 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.93, 0.02, 0.05]), array([0.93, 0.02, 0.05]), array([0.94, 0.06, 0.  ]), array([0.93, 0.07, 0.  ]), array([0.94, 0.05, 0.01]), array([0.94, 0.06, 0.  ]), -5.846913483015742, 11.168092473854982, 11.112929602424952)


# a = 0

In [17]:
a = 0.0
num_params=19
num_sims = 100

In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.49 & 0.49 & 0.99 & 0.98 & 0.98 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.51 & 0.51 & 0.00 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.49, 0.  , 0.51]), array([0.49, 0.  , 0.51]), array([0.99, 0.01, 0.  ]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.99, 0.01, 0.  ]), -8.78190951510698, 3.4735315903000705, 4.240917490653393)


In [19]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.41 & 0.41 & 0.99 & 0.99 & 0.99 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.59 & 0.59 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.41, 0.  , 0.59]), array([0.41, 0.  , 0.59]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), -8.926101859066554, 3.2739175110108816, 4.383246340557531)


# evidence of power

In [20]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9
num_sims = 100

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [21]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.84 & 0.84 & 0.48 & 0.51 & 0.49 & 0.60   \\
Model 1 & 0.16 & 0.16 & 0.52 & 0.49 & 0.51 & 0.40   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.84, 0.16, 0.  ]), array([0.84, 0.16, 0.  ]), array([0.48, 0.52, 0.  ]), array([0.51, 0.49, 0.  ]), array([0.49, 0.51, 0.  ]), array([0.6, 0.4, 0. ]), 6.215476741598551, 5.373122666140737, 4.974366686244583)


In [22]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.79 & 0.79 & 0.52 & 0.51 & 0.47 & 0.54   \\
Model 1 & 0.21 & 0.21 & 0.48 & 0.49 & 0.53 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.79, 0.21, 0.  ]), array([0.79, 0.21, 0.  ]), array([0.52, 0.48, 0.  ]), array([0.51, 0.49, 0.  ]), array([0.47, 0.53, 0.  ]), array([0.54, 0.46, 0.  ]), 7.219461457623967, 4.475706382469549, 5.372532759880559)


In [23]:
a1,a2 = 0.00,np.sqrt(1.09-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.12 & 0.12 & 0.50 & 0.53 & 0.48 & 0.57   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.88 & 0.88 & 0.50 & 0.47 & 0.52 & 0.43   \\
\hline
\end{tabular}
(array([0.12, 0.  , 0.88]), array([0.12, 0.  , 0.88]), array([0.5, 0. , 0.5]), array([0.53, 0.  , 0.47]), array([0.48, 0.  , 0.52]), array([0.57, 0.  , 0.43]), -14.339085274935174, 5.818415056232851, 5.2509620911971435)


In [24]:
a1,a2 =  0.00,np.sqrt(1.09**.5-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.16 & 0.16 & 0.54 & 0.52 & 0.52 & 0.59   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.84 & 0.84 & 0.46 & 0.48 & 0.48 & 0.41   \\
\hline
\end{tabular}
(array([0.16, 0.  , 0.84]), array([0.16, 0.  , 0.84]), array([0.54, 0.  , 0.46]), array([0.52, 0.  , 0.48]), array([0.52, 0.  , 0.48]), array([0.59, 0.  , 0.41]), -13.545118893551194, 4.6864122193996245, 5.227885131513371)
